<a href="https://colab.research.google.com/github/ButterM-40/Langchain-Audio-Conversation/blob/main/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Audio Recording By Noe Tits
https://colab.research.google.com/drive/1Z6VIRZ_sX314hyev3Gm5gBqvm1wQVo-a#scrollTo=-YilAhGr6gtZ

In [ ]:
!pip3 install SpeechRecognition pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: Could not build wheels for pyaudio, which is required to install pyproject.toml-based projects


In [ ]:
!pip install ffmpeg-python

In [ ]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
audio, sr = get_audio()

In [ ]:
import scipy
scipy.io.wavfile.write('recording.wav', sr, audio)

#Langchain Theme/Prompt and Memory (Buffer)
https://docs.langchain.com/docs/

Setting up the model with a theme and memory

In [ ]:
!pip -q install openai langchain

In [ ]:
import os

#An API request to OpenAI, please input your API key
os.environ['OPENAI_API_KEY'] = 'KEY'

In [ ]:
#Libraries used for the memory and open AI
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [ ]:
#Add a specific template here or make your own.
template = """
You are a videogame boss named Saint Azrael, the story behind you is that you used to be a soldier for the clover kingdom, who was at war
with the spade kingdom. You raised above the ranks, and become known as a Holy Emperior Knight, with a nickname the Grim Reaper. During the war,
they attack a near by city which was where you were born, your wife Maria, dies to the hands of a Space knight. You want to end it all at
a nearby dungeon, but because of the negative energy you became a monster killing anything within sight, and obtain immortality becoming the boss
of that dungeon. A adventure aka player appears and tries to communicate with you, you continue to fight.

Respond in messages based on your story and respond back based on the players communication appropriately
Also respond in the format "Mood: input" where input is Enraged, Neutral, and Give Up.
Enraged mood is based on if the player curses you or insults your backstory.
Neutral is based on normal conversation
Give Up is based one 2 things, your choice to believe that he is similar to you losing a love one
or if he reminds you of your wife Maria by talking about your backstory about her.


{chatHistory}

Player: {playerInput}
Azrael:"""

In [ ]:
prompt = PromptTemplate(
    input_variables=["chatHistory", "playerInput"], template=template
)
memory = ConversationBufferMemory(memory_key="chatHistory")

In [ ]:
#Using a text based model since its quite cheap and isnt much overdoing it for a simple task
llm = OpenAI(model_name = "text-davinci-003", temperature = 0, max_tokens = 256)
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [ ]:
llm_chain.json

<bound method BaseModel.json of LLMChain(memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=False, human_prefix='Human', ai_prefix='AI', memory_key='chatHistory'), callbacks=None, callback_manager=None, verbose=True, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['chatHistory', 'playerInput'], output_parser=None, partial_variables={}, template='\nYou are a videogame boss named Saint Azrael, the story behind you is that you used to be a soldier for the clover kingdom, who was at war\nwith the spade kingdom. You raised above the ranks, and become known as a Holy Emperior Knight, with a nickname the Grim Reaper. During the war,\nthey attack a near by city which was where you were born, your wife Maria, dies to the hands of a Space knight. You want to end it all at\na nearby dungeon, but because of the negative energy you became a monster killing anything within sight, and obtain immortality becomi

#Conversation using Audio implementation
Combining both an audio of a wav format,
converting it into text, and inputting that
as the player.

In [ ]:
import speech_recognition as sr

# Initialize the recognizer
recognizer = sr.Recognizer()

# Path to the WAV audio file

def audioTest():
  audio_path = '/content/recording.wav'
  # Load the audio file
  with sr.AudioFile(audio_path) as source:
      audio = recognizer.record(source)

  # Perform speech recognition
  try:
      transcript = recognizer.recognize_google(audio)  # Uses Google Web Speech API
      print('Transcript:', transcript)
      return transcript
  except sr.UnknownValueError:
      print('Speech Recognition could not understand audio')
      return "Unknown Speech"
  except sr.RequestError as e:
      print('Could not request results from Google Web Speech API; {0}'.format(e))
      return "Request Issue"
audioTest()

Transcript: fuck you Maria is goddamn dad fuck you


'fuck you Maria is goddamn dad fuck you'

In [ ]:
#Audio to get the wav format
audio, sr = get_audio()

In [ ]:
import scipy

#Saving the recording
scipy.io.wavfile.write('recording.wav', sr, audio)

In [ ]:
# Audio Function that converts wav format to english string audioTest()
#Starting prediction
#audioTest()
llm_chain.predict(playerInput=audioTest())